In [18]:
import networkx as nx
from similarity_measures import simplified_dmeasure as dm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nibabel as nib
import os
from matplotlib.colors import LinearSegmentedColormap

In [16]:
base_path = '/Users/ahmet/Desktop/Study/second_semester/ds_project/BSNIP/pconn'

In [14]:
pconn_paths = os.listdir('/Users/ahmet/Desktop/Study/second_semester/ds_project/BSNIP/pconn')

In [15]:
session_ids = [path.split('.')[0] for path in pconn_paths]

In [11]:
def dists(Gs, measure):
    
    Ds = [[0] * len(Gs) for _ in range(len(Gs))]

    for i in range(len(Gs)):
        for j in range(i, len(Gs)):
            Ds[i][j] = measure(Gs[i], Gs[j])
            Ds[j][i] = Ds[i][j]

    return Ds


def build_network(data, df, visualize = True):
    # Create a graph from the data
    # This example assumes 'data' is a square matrix where data[i][j] represents the connection
    # strength between region i and region j. Your data's structure may vary.
    G = nx.Graph()
    
    # Add nodes with the brain region names or indices if you don't have names
    num_regions = data.shape[0]
    G.add_nodes_from(range(num_regions))
    
    # Add edges based on connectivity data
    # Here we're simply adding an edge for every non-zero connection
    for i in range(num_regions):
        # Calculate the standard deviation of the specified row
        std_dev = df.iloc[i].std()
        threshold = 2*std_dev
        for j in range(i+1, num_regions):  # Ensure i < j to avoid duplicating edges
            if abs(data[i, j]) > threshold:  # Assuming 0 means no connection
                G.add_edge(i, j, weight=data[i, j])
    
    if visualize:
        # You can also visualize the graph using matplotlib or similar libraries
        # This step requires matplotlib to be installed (`pip install matplotlib`)
        nx.draw(G, with_labels=True)
        plt.show()
        
    return G

def read_pconn(path):
    
    # Load the .pconn.nii file
    img = nib.load(path)
    
    # Extract the data array from the image
    # The data might need to be processed or reshaped depending on its structure
    data = img.get_fdata()

    df = pd.DataFrame(data)
    
    return data, df

def map(Gs, Ds, label):
    fig = plt.figure()

    plt.imshow(Ds, cmap=LinearSegmentedColormap.from_list(
        '', ['yellow', 'gray', 'white']))

    for i in range(len(Gs)):
        for j in range(len(Gs)):
            plt.text(j, i, "{:.2f}".format(
                Ds[i][j]), ha='center', va='center', fontsize=4)

    plt.title(label)
    plt.xticks(ticks=[])
    plt.yticks(ticks=range(len(Gs)), labels=[G.name for G in Gs], fontsize=7)

    clb = plt.colorbar()
    clb.ax.tick_params(labelsize=7)

    fig.savefig(label + ".pdf", bbox_inches='tight')
    plt.close(fig)

In [17]:
Gs = []
for session_id, pconn_path in zip(session_ids, pconn_paths):
    path = os.path.join(base_path, pconn_path)
    data, df = read_pconn(path)
    G = build_network(data, df, visualize = False)
    Gs.append(G)

In [21]:
Ds = dists(Gs, dm.dmeasure)

KeyboardInterrupt: 